In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pycountry
import numpy as np
from scipy import stats


MEGAMERGE is the culmination of a weeks effort of cleaning and combining data. Making sure everything clicks and workd in perfect harmony wih no null values at the end to be able to do machine learning.

In [2]:
# Load the datasets
gender_gap_education = pd.read_csv('/Users/javi/Desktop/Ironhack/Entregas_de_github/Final-Project-at-Ironhack/Data/Cleaned/gender-gap-education-levels_cleanded.csv')
birth_rate = pd.read_csv('/Users/javi/Desktop/Ironhack/Entregas_de_github/Final-Project-at-Ironhack/Data/Cleaned/birth_rate_first_clean.csv')
death_rate = pd.read_csv('/Users/javi/Desktop/Ironhack/Entregas_de_github/Final-Project-at-Ironhack/Data/Cleaned/death rate.csv')
gdp_per_capita = pd.read_csv('/Users/javi/Desktop/Ironhack/Entregas_de_github/Final-Project-at-Ironhack/Data/Cleaned/Gpd_per_capita_first_clean.csv')
population_growth = pd.read_csv('/Users/javi/Desktop/Ironhack/Entregas_de_github/Final-Project-at-Ironhack/Data/Cleaned/population-growth-rate-vs-median-age.csv')
world_population = pd.read_csv('/Users/javi/Desktop/Ironhack/Entregas_de_github/Final-Project-at-Ironhack/Data/Cleaned/world_population.csv')

In [3]:
# Merge dataframes on 'Country Name' and 'Year'
merged_df = gender_gap_education.merge(birth_rate, on=['Country Name', 'Year'], how='outer')
merged_df = merged_df.merge(death_rate, on=['Country Name', 'Year'], how='outer')
merged_df = merged_df.merge(gdp_per_capita, on=['Country Name', 'Year'], how='outer')
merged_df = merged_df.merge(population_growth, on=['Country Name', 'Year'], how='outer')
merged_df = merged_df.merge(world_population, on=['Country Name', 'Year'], how='outer')

# Check for missing values
missing_values = merged_df.isnull().sum()

# Check for consistent data types
data_types = merged_df.dtypes

# Display results
missing_values, data_types


(Country Name                  0
 Year                          0
 Tertiary Enrolment (F)     5040
 Tertiary Enrolment (M)     5040
 Secondary Enrolment (M)    5040
 Secondary Enrolment (F)    5040
 Primary Enrolment (F)      5040
 Primary Enrolment (M)      5040
 Birth Rate                 3824
 Death Rate                 3822
 GDP per Capita             4992
 Natural Growth Rate        3864
 Median Age                 3864
 Total Population           3402
 dtype: int64,
 Country Name                object
 Year                         int64
 Tertiary Enrolment (F)     float64
 Tertiary Enrolment (M)     float64
 Secondary Enrolment (M)    float64
 Secondary Enrolment (F)    float64
 Primary Enrolment (F)      float64
 Primary Enrolment (M)      float64
 Birth Rate                 float64
 Death Rate                 float64
 GDP per Capita             float64
 Natural Growth Rate        float64
 Median Age                 float64
 Total Population           float64
 dtype: object)

In [4]:
# Define regions including European Union and Non-EU Europe and assign countries to regions
region_mapping = {
    'Asia': ['Afghanistan', 'Armenia', 'Azerbaijan', 'Bahrain', 'Bangladesh', 'Bhutan', 'Brunei', 'Cambodia', 'China', 'Cyprus', 'Georgia', 'India', 'Indonesia', 'Iran', 'Iraq', 'Israel', 'Japan', 'Jordan', 'Kazakhstan', 'Kuwait', 'Kyrgyzstan', 'Laos', 'Lebanon', 'Malaysia', 'Maldives', 'Mongolia', 'Myanmar', 'Nepal', 'North Korea', 'Oman', 'Pakistan', 'Palestine', 'Philippines', 'Qatar','Russia', 'Saudi Arabia', 'Singapore', 'South Korea', 'Sri Lanka', 'Syria', 'Tajikistan', 'Thailand', 'Timor-Leste', 'Turkmenistan', 'United Arab Emirates', 'Uzbekistan', 'Vietnam', 'Yemen'],
    'Africa': ['Algeria', 'Angola', 'Benin', 'Botswana', 'Burkina Faso', 'Burundi', 'Cabo Verde', 'Cameroon', 'Central African Republic', 'Chad', 'Comoros', 'Congo', 'Cote d\'Ivoire', 'Djibouti', 'Egypt', 'Equatorial Guinea', 'Eritrea', 'Eswatini', 'Ethiopia', 'Gabon', 'Gambia', 'Ghana', 'Guinea', 'Guinea-Bissau', 'Kenya', 'Lesotho', 'Liberia', 'Libya', 'Madagascar', 'Malawi', 'Mali', 'Mauritania', 'Mauritius', 'Morocco', 'Mozambique', 'Namibia', 'Niger', 'Nigeria', 'Rwanda', 'Sao Tome and Principe', 'Senegal', 'Seychelles', 'Sierra Leone', 'Somalia', 'South Africa', 'South Sudan', 'Sudan', 'Tanzania', 'Togo', 'Tunisia', 'Uganda', 'Zambia', 'Zimbabwe'],
    'European Union': ['Austria', 'Belgium', 'Bulgaria', 'Croatia', 'Cyprus', 'Czech Republic', 'Denmark', 'Estonia', 'Finland', 'France', 'Germany', 'Greece', 'Hungary', 'Ireland', 'Italy', 'Latvia', 'Lithuania', 'Luxembourg', 'Malta', 'Netherlands', 'Poland', 'Portugal', 'Romania', 'Slovakia', 'Slovenia', 'Spain', 'Sweden'],
    'Non-EU Europe': ['Albania', 'Andorra', 'Belarus', 'Bosnia and Herzegovina', 'Iceland', 'Kosovo', 'Moldova', 'Monaco', 'Montenegro', 'North Macedonia', 'Norway', 'San Marino', 'Serbia', 'Switzerland', 'Ukraine', 'United Kingdom', 'Vatican'],
    'Oceania': ['Australia', 'Fiji', 'Kiribati', 'Marshall Islands', 'Micronesia', 'Nauru', 'New Zealand', 'Palau', 'Papua New Guinea', 'Samoa', 'Solomon Islands', 'Tonga', 'Tuvalu', 'Vanuatu'],
    'North America': ['Antigua and Barbuda', 'Bahamas', 'Barbados', 'Belize', 'Canada', 'Costa Rica', 'Cuba', 'Dominica', 'Dominican Republic', 'El Salvador', 'Grenada', 'Guatemala', 'Haiti', 'Honduras', 'Jamaica', 'Mexico', 'Nicaragua', 'Panama', 'Saint Kitts and Nevis', 'Saint Lucia', 'Saint Vincent and the Grenadines', 'Trinidad and Tobago', 'United States'],
    'South America': ['Argentina', 'Bolivia', 'Brazil', 'Chile', 'Colombia', 'Ecuador', 'Guyana', 'Paraguay', 'Peru', 'Suriname', 'Uruguay', 'Venezuela'],
    'World': ['World']
}

# Flatten the region_mapping dictionary to reverse lookup the region for a country
country_to_region = {country: region for region, countries in region_mapping.items() for country in countries}

# Assign regions to countries in the merged_df
merged_df['Region'] = merged_df['Country Name'].apply(lambda x: country_to_region.get(x, 'Various'))

# Verify the new 'Region' column
merged_df.head()


,Country Name,Year,Tertiary Enrolment (F),Tertiary Enrolment (M),Secondary Enrolment (M),Secondary Enrolment (F),Primary Enrolment (F),Primary Enrolment (M),Birth Rate,Death Rate,GDP per Capita,Natural Growth Rate,Median Age,Total Population,Region
0,Afghanistan,1980,0.500000,1.860000,13.760000,3.520000,11.000000,46.060000,7.59,24.21,NaN,2.63,15.9,12486631.0,Asia
1,Afghanistan,1981,0.520303,2.150378,14.289395,3.848606,11.207511,45.501156,7.57,23.61,NaN,2.67,15.8,11155195.0,Asia
2,Afghanistan,1982,0.540607,2.440757,14.818789,4.177212,11.415022,44.942312,7.55,25.62,NaN,2.45,15.7,10088289.0,Asia
3,Afghanistan,1983,0.560910,2.731135,15.348184,4.505818,11.622532,44.383468,7.54,25.11,NaN,2.50,15.6,9951449.0,Asia
4,Afghanistan,1984,0.581213,3.021513,15.877578,4.834425,11.830043,43.824625,7.51,30.03,NaN,2.02,15.5,10243686.0,Asia


In [5]:

# Ensure the updated_regions dictionary includes the simplified names
updated_regions = {
    "South Korea": "Asia",
    "North Korea": "Asia",
    "Taiwan": "Asia",
    "Vietnam": "Asia",
    "American Samoa": "Oceania",
    "Anguilla": "North America",
    "Aruba": "North America",
    "Bermuda": "North America",
    "British Virgin Islands": "North America",
    "Brunei Darussalam": "Asia",
    "Cayman Islands": "North America",
    "Cook Islands": "Oceania",
    "Czechia": "European Union",
    "Faroe Islands": "Non-EU Europe",
    "French Guiana": "South America",
    "French Polynesia": "Oceania",
    "Gibraltar": "Non-EU Europe",
    "Greenland": "North America",
    "Guadeloupe": "North America",
    "Guam": "Oceania",
    "Guernsey": "Non-EU Europe",
    "Hong Kong": "Asia",
    "Isle of Man": "Non-EU Europe",
    "Jersey": "Non-EU Europe",
    "Kyrgyz Republic": "Asia",
    "Liechtenstein": "Non-EU Europe",
    "Macao": "Asia",
    "Martinique": "North America",
    "Mayotte": "Africa",
    "Montserrat": "North America",
    "New Caledonia": "Oceania",
    "Niue": "Oceania",
    "Northern Mariana Islands": "Oceania",
    "Puerto Rico": "North America",
    "Russian Federation": "Asia",
    "Saint Martin (French part)": "North America",
    "Saint Pierre and Miquelon": "North America",
    "Sint Maarten (Dutch part)": "North America",
    "Slovak Republic": "European Union",
    "Syrian Arab Republic": "Asia",
    "Taiwan": "Asia",
    "Tokelau": "Oceania",
    "Turkey": "Asia",
    "Turks and Caicos Islands": "North America",
    "Viet Nam": "Asia",
    "Wallis and Futuna": "Oceania",
    "Western Sahara": "Africa"
}

# Update the region in the dataframe
merged_df['Region'] = merged_df['Country Name'].map(updated_regions).fillna(merged_df['Region'])

In [6]:
# Define the new function to directly rename the specified countries
def rename_countries(df):
    rename_mapping = {
        'Korea, Rep.': 'South Korea',
        'Korea, Dem. People’s Rep.': 'North Korea',
        "Korea, Dem. People's Rep.": 'North Korea',
        'Taiwan': 'Taiwan',
        'Viet Nam': 'Vietnam',
        'Vietnam': 'Vietnam',
        'Syrian Arab Republic' : 'Syria'
    }
    df['Country Name'] = df['Country Name'].replace(rename_mapping)
    return df

# Apply the renaming function to the merged_df
combined_data = rename_countries(merged_df)

In [7]:
# Drop the specified rows
countries_to_drop = [
    'Europe & Central Asia',
    'Europe & Central Asia (excluding high income)', 'Europe and Central Asia (WB)',
    'Euro area', 'Anguilla', 'Europe (UN)', 'European Union', 'European Union (27)'
]

filtered_data = merged_df[~merged_df['Country Name'].isin(countries_to_drop)]

# Verify the rows are dropped
filtered_data_check = filtered_data[filtered_data['Country Name'].isin(countries_to_drop)]

In [8]:
# Replace specific country names with "European Union"
countries_to_rename = ['Europe (UN)', 'European Union', 'European Union (27)']
filtered_data['Country Name'] = filtered_data['Country Name'].replace(countries_to_rename, 'European Union')

# Verify the replacement
replacement_check = filtered_data[filtered_data['Country Name'] == 'European Union']

/var/folders/23/ykjhg9v57tn3wfmhgxtgj1s80000gn/T/ipykernel_65332/851706039.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['Country Name'] = filtered_data['Country Name'].replace(countries_to_rename, 'European Union')


In [9]:
def merge_similar_names(combined_data, target_name, similar_names, target_region):
    # Replace specific country names with the target name
    combined_data['Country Name'] = combined_data['Country Name'].replace(similar_names, target_name)
    
    # Filter by region for the target name where region is 'Various' or a continent
    continents = ['Africa', 'Asia', 'European Union', 'Non-EU Europe', 'North America', 'Oceania', 'South America']
    filtered_data = combined_data[(combined_data['Country Name'] == target_name) & (combined_data['Region'].isin(['Various'] + continents))]
    
    # Group and aggregate by year
    aggregated_data = filtered_data.groupby('Year').agg({
        'Tertiary Enrolment (F)': 'mean',
        'Tertiary Enrolment (M)': 'mean',
        'Secondary Enrolment (M)': 'mean',
        'Secondary Enrolment (F)': 'mean',
        'Primary Enrolment (F)': 'mean',
        'Primary Enrolment (M)': 'mean',
        'Birth Rate': 'mean',
        'Death Rate': 'mean',
        'GDP per Capita': 'mean',
        'Natural Growth Rate': 'mean',
        'Median Age': 'mean',
        'Total Population': 'sum'
    }).reset_index()
    
    # Add 'Country Name' and 'Region' columns to the aggregated combined_data
    aggregated_data['Country Name'] = target_name
    aggregated_data['Region'] = target_region
    
    # Separate the rest of the combined_data excluding the target name with 'Various' or continent regions
    other_data = combined_data[~((combined_data['Country Name'] == target_name) & (combined_data['Region'].isin(['Various'] + continents)))]
    
    # Combine the aggregated combined_data with the rest of the dataframe
    combined_data = pd.concat([other_data, aggregated_data], ignore_index=True)
    
    return combined_data

# List of countries to merge
countries_to_merge = [
    {'target_name': 'Vietnam', 'similar_names': ['Viet Nam'], 'target_region': 'Asia'},
    {'target_name': 'Cape Verde', 'similar_names': ['Cabo Verde'], 'target_region': 'Africa'},
    {'target_name': 'Saint Kitts and Nevis', 'similar_names': ['St. Kitts and Nevis'], 'target_region': 'North America'},
    {'target_name': 'Saint Lucia', 'similar_names': ['St. Lucia'], 'target_region': 'North America'},
    {'target_name': 'Saint Martin (French part)', 'similar_names': ['St. Martin (French part)'], 'target_region': 'North America'},
    {'target_name': 'Saint Vincent and the Grenadines', 'similar_names': ['St. Vincent and the Grenadines'], 'target_region': 'North America'},
    {'target_name': 'Venezuela', 'similar_names': ['Venezuela, RB'], 'target_region': 'South America'},
    {'target_name': 'Yemen', 'similar_names': ['Yemen, Rep.'], 'target_region': 'Asia'},
    {'target_name': 'Brunei', 'similar_names': ['Brunei Darussalam'], 'target_region': 'Asia'},
    {'target_name': 'Curaçao', 'similar_names': ['Curacao'], 'target_region': 'South America'},
    {'target_name': 'Czech Republic', 'similar_names': ['Czechia'], 'target_region': 'European Union'},
    {'target_name': 'Congo, The Democratic Republic of the', 'similar_names': ['Democratic Republic of Congo', 'Congo, Dem. Rep.'], 'target_region': 'Africa'},
    {'target_name': 'Congo', 'similar_names': ['Congo, Rep.'], 'target_region': 'Africa'},
    {'target_name': 'Cook Islands', 'similar_names': ['Cook Islands'], 'target_region': 'Oceania'},
    {'target_name': 'Aruba', 'similar_names': ['Aruba'], 'target_region': 'South America'},
    {'target_name': 'Syria', 'similar_names': ['Syrian Arab Republic'], 'target_region': 'Asia'},
    {'target_name': 'Russia', 'similar_names': ['Russian Federation'], 'target_region': 'Asia'},
    {'target_name': 'South Korea', 'similar_names': ['Korea, Rep.'], 'target_region': 'Asia'},
    {'target_name': 'North Korea', 'similar_names': ['Korea, Dem. People’s Rep.'], 'target_region': 'Asia'},
    {'target_name': 'Iran', 'similar_names': ['Iran, Islamic Rep.'], 'target_region': 'Asia'},
    {'target_name': 'Laos', 'similar_names': ['Lao PDR'], 'target_region': 'Asia'},
    {'target_name': 'Myanmar', 'similar_names': ['Burma'], 'target_region': 'Asia'},
    {'target_name': 'North Macedonia', 'similar_names': ['Macedonia'], 'target_region': 'Non-EU Europe'},
    {'target_name': 'Tanzania', 'similar_names': ['United Republic of Tanzania'], 'target_region': 'Africa'},
    {'target_name': 'Ivory Coast', 'similar_names': ['Côte d\'Ivoire'], 'target_region': 'Africa'},
    {'target_name': 'North Korea', 'similar_names': ["Korea, Dem. People's Rep."], 'target_region': 'Asia'},
    {'target_name': 'Bahamas', 'similar_names': ['Bahamas, The'], 'target_region': 'North America'},
    {'target_name': 'Jersey', 'similar_names': ['Channel Islands'], 'target_region': 'Non-EU Europe'},
    {'target_name': 'British Virgin Islands', 'similar_names': ['United States Virgin Islands', 'Virgin Islands (U.S.)'], 'target_region': 'North America'},
    {'target_name': 'Egypt', 'similar_names': ['Egypt, Arab Rep.'], 'target_region': 'Africa'},
    {'target_name': 'Gambia', 'similar_names': ['Gambia, The'], 'target_region': 'Africa'},
    {'target_name': 'Hong Kong', 'similar_names': ['Hong Kong SAR, China'], 'target_region': 'Asia'},
    {'target_name': 'Macao', 'similar_names': ['Macao SAR, China'], 'target_region': 'Asia'},
    {'target_name': 'Turkey', 'similar_names': ['Turkiye'], 'target_region': 'Asia'},
    {'target_name': 'Timor-Leste', 'similar_names': ['East Timor'], 'target_region': 'Asia'},
    {'target_name': 'Micronesia', 'similar_names': ['Micronesia (country)', 'Micronesia, Fed. Sts.'], 'target_region': 'Oceania'},
    {'target_name': 'Slovakia', 'similar_names': ['Slovak Republic'], 'target_region': 'European Union'},
    {'target_name': 'Kyrgyzstan', 'similar_names': ['Kyrgyz Republic'], 'target_region': 'Asia'},
    
    # Add more countries as needed
]

# Apply the merging function for each country in the list
for country in countries_to_merge:
    combined_data = merge_similar_names(combined_data, country['target_name'], country['similar_names'], country['target_region'])


combined_data.tail()


,Country Name,Year,Tertiary Enrolment (F),Tertiary Enrolment (M),Secondary Enrolment (M),Secondary Enrolment (F),Primary Enrolment (F),Primary Enrolment (M),Birth Rate,Death Rate,GDP per Capita,Natural Growth Rate,Median Age,Total Population,Region
13309,Kyrgyzstan,2017,47.51968,38.29963,83.38251,82.93139,90.741003,91.297901,3.00,5.4,1242.77,2.11,23.4,6198200.0,Asia
13310,Kyrgyzstan,2018,46.73536,36.00659,84.64377,84.12084,89.277466,89.984567,3.30,5.2,1308.14,2.14,23.5,6322800.0,Asia
13311,Kyrgyzstan,2019,47.44449,37.38893,84.64377,84.12084,89.277466,89.984567,3.30,5.2,1451.52,2.08,23.6,6456200.0,Asia
13312,Kyrgyzstan,2020,51.72928,41.36852,84.64377,84.12084,89.277466,89.984567,3.00,6.1,1256.93,1.86,23.6,6579900.0,Asia
13313,Kyrgyzstan,2021,59.41218,47.73265,84.64377,84.12084,89.277466,89.984567,2.89,5.8,1365.51,1.79,23.7,6773400.0,Asia


In [10]:
excluded_regions = ['World', 'Various']
filtered_data = combined_data[~combined_data['Region'].isin(excluded_regions)]
unique_country_count = filtered_data['Country Name'].nunique()

unique_country_count

231

In [11]:
# Filter out the entries from the "Various" region
combined_data_no_various = combined_data[combined_data['Region'] != 'Various']
combined_data_no_various.isna().sum()

Country Name                  0
Year                          0
Tertiary Enrolment (F)     1050
Tertiary Enrolment (M)     1050
Secondary Enrolment (M)    1050
Secondary Enrolment (F)    1050
Primary Enrolment (F)      1050
Primary Enrolment (M)      1050
Birth Rate                 1010
Death Rate                 1008
GDP per Capita             1842
Natural Growth Rate           0
Median Age                    0
Total Population            630
Region                        0
dtype: int64

In [12]:
# List of countries to drop
countries_to_drop = [
    "French Guiana", "Guadeloupe", "Guernsey", "Martinique", "Mayotte", 
    "Montserrat", "Niue", "Saint Pierre and Miquelon", "Tokelau", 
    "Western Sahara", "Cook Islands", "Wallis and Futuna" 
]

# Drop the specified countries
combined_data_no_various_dropped_small = combined_data_no_various[~combined_data_no_various['Country Name'].isin(countries_to_drop)]
combined_data_no_various_dropped_small.nunique()

Country Name                220
Year                         42
Tertiary Enrolment (F)     6269
Tertiary Enrolment (M)     6296
Secondary Enrolment (M)    5361
Secondary Enrolment (F)    5334
Primary Enrolment (F)      5423
Primary Enrolment (M)      5437
Birth Rate                  749
Death Rate                 1781
GDP per Capita             7535
Natural Growth Rate         546
Median Age                  397
Total Population           9057
Region                        8
dtype: int64

In [13]:
#save the data cleanly in cleaned file
combined_data_no_various_dropped_small.to_csv('/Users/javi/Desktop/Ironhack/Entregas_de_github/Final-Project-at-Ironhack/Data/Cleaned/MEGAMERGE_cleaned.csv',index=False, encoding='utf-8')

In [14]:
#Filling missing values using other Dataframes as a reference.
url_worldbank_gdp = ('/Users/javi/Desktop/Ironhack/Entregas_de_github/Final-Project-at-Ironhack/Data/Raw data/World demographic/Gdppercapita worldbank + imf/gdp per capita.csv')
url_imf_gpd = ('/Users/javi/Desktop/Ironhack/Entregas_de_github/Final-Project-at-Ironhack/Data/Raw data/World demographic/Gdppercapita worldbank + imf/imf-dm-export-20240530.csv')
megammerge_file_path = ('/Users/javi/Desktop/Ironhack/Entregas_de_github/Final-Project-at-Ironhack/Data/Cleaned/MEGAMERGE_cleaned.csv')

csv_data_new = pd.read_csv(url_worldbank_gdp)
csv_data_imf = pd.read_csv(url_imf_gpd)
megammerge_data = pd.read_csv(megammerge_file_path)

In [15]:
# Convert year columns in csv_data_new and csv_data_imf to numeric, replacing errors with NaN
for year in range(1980, 2022):
    csv_data_new[str(year)] = pd.to_numeric(csv_data_new[str(year)], errors='coerce')
    csv_data_imf[str(year)] = pd.to_numeric(csv_data_imf[str(year)], errors='coerce')

# Create a dictionary for easy access to GDP data from csv_data_new and csv_data_imf
def create_gdp_dict(data, country_col, start_year_col):
    gdp_dict = {}
    for index, row in data.iterrows():
        country = row[country_col]
        if country not in gdp_dict:
            gdp_dict[country] = {}
        for year in range(1980, 2022):
            if str(year) in row and pd.notna(row[str(year)]):
                gdp_dict[country][year] = row[str(year)]
    return gdp_dict

gdp_dict_csv_new = create_gdp_dict(csv_data_new, 'Country Name', '1980')
gdp_dict_imf = create_gdp_dict(csv_data_imf, 'GDP per capita, current prices (Purchasing power parity; international dollars per capita)', '1980')

# Fill missing GDP per Capita in MEGAMERGE_cleaned using data from both sources
for index, row in megammerge_data.iterrows():
    country = row['Country Name']
    year = int(row['Year'])
    if pd.isna(row['GDP per Capita']):
        gdp_values = []
        if country in gdp_dict_csv_new and year in gdp_dict_csv_new[country]:
            gdp_values.append(gdp_dict_csv_new[country][year])
        if country in gdp_dict_imf and year in gdp_dict_imf[country]:
            gdp_values.append(gdp_dict_imf[country][year])
        if gdp_values:
            megammerge_data.at[index, 'GDP per Capita'] = sum(gdp_values) / len(gdp_values)


In [16]:
# Fill missing GDP per Capita for 'Virgin Islands, British' using data from 'Virgin Islands (U.S.)'
reference_country = 'Virgin Islands (U.S.)'
target_country = 'Virgin Islands, British'

# Extract GDP data for the reference country
gdp_reference = {}
for index, row in csv_data_new.iterrows():
    if row['Country Name'] == reference_country:
        for year in range(1980, 2022):
            if pd.notna(row[str(year)]):
                gdp_reference[year] = row[str(year)]

for index, row in csv_data_imf.iterrows():
    if row['GDP per capita, current prices (Purchasing power parity; international dollars per capita)'] == reference_country:
        for year in range(1980, 2022):
            if pd.notna(row[str(year)]):
                gdp_reference[year] = row[str(year)]

# Fill missing values in MEGAMERGE_cleaned for the target country
for index, row in megammerge_data.iterrows():
    country = row['Country Name']
    year = int(row['Year'])
    if country == target_country and pd.isna(row['GDP per Capita']) and year in gdp_reference:
        megammerge_data.at[index, 'GDP per Capita'] = gdp_reference[year]


In [17]:
megammerge_data.isna().sum()

Country Name                 0
Year                         0
Tertiary Enrolment (F)     546
Tertiary Enrolment (M)     546
Secondary Enrolment (M)    546
Secondary Enrolment (F)    546
Primary Enrolment (F)      546
Primary Enrolment (M)      546
Birth Rate                 506
Death Rate                 504
GDP per Capita             925
Natural Growth Rate          0
Median Age                   0
Total Population           168
Region                       0
dtype: int64

In [18]:
# Define a function to create a complete set of years for interpolation
def complete_years(df, start_year, end_year):
    idx = pd.MultiIndex.from_product([df['Country Name'].unique(), range(start_year, end_year+1)], names=['Country Name', 'Year'])
    df = df.set_index(['Country Name', 'Year']).reindex(idx).reset_index()
    return df

# Apply the function to the filtered data
complete_data = complete_years(megammerge_data, 1980, 2021)


In [19]:
interpolated_data_complete = complete_data.groupby('Country Name', group_keys=True).apply(lambda x: x.sort_values('Year').interpolate(method='linear')).reset_index(drop=True)
filled_data = interpolated_data_complete.groupby('Country Name').apply(lambda x: x.sort_values('Year').fillna(method='ffill').fillna(method='bfill')).reset_index(drop=True)


/var/folders/23/ykjhg9v57tn3wfmhgxtgj1s80000gn/T/ipykernel_65332/3376950692.py:1: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  interpolated_data_complete = complete_data.groupby('Country Name', group_keys=True).apply(lambda x: x.sort_values('Year').interpolate(method='linear')).reset_index(drop=True)
/var/folders/23/ykjhg9v57tn3wfmhgxtgj1s80000gn/T/ipykernel_65332/3376950692.py:1: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  interpolated_data_complete = complete_data.groupby('Country Name', group_keys=True).apply(lambda x: x.sort_values('Year').interpolate(method='linear')).reset_index(drop=True)
/var/folders/23/ykjhg9v57tn3wfmhgxtgj1s80000gn/T/ipykernel_65332/3376950692.py:1: FutureWarning: DataFrame.interpolate with object dtype is de

In [20]:
interpolated_data_complete.isna().sum()

Country Name                 0
Year                         0
Tertiary Enrolment (F)     546
Tertiary Enrolment (M)     546
Secondary Enrolment (M)    546
Secondary Enrolment (F)    546
Primary Enrolment (F)      546
Primary Enrolment (M)      546
Birth Rate                 506
Death Rate                 504
GDP per Capita             874
Natural Growth Rate          0
Median Age                   0
Total Population           168
Region                       0
dtype: int64

In [21]:
filtered_df = merged_df[merged_df['Country Name'] == 'China']

In [22]:
filtered_df

,Country Name,Year,Tertiary Enrolment (F),Tertiary Enrolment (M),Secondary Enrolment (M),Secondary Enrolment (F),Primary Enrolment (F),Primary Enrolment (M),Birth Rate,Death Rate,GDP per Capita,Natural Growth Rate,Median Age,Total Population,Region
2310,China,1980,0.500000,2.030000,55.110000,36.260000,82.220000,78.780000,2.74,6.34,312.02,1.47,20.8,9.812350e+08,Asia
2311,China,1981,0.640961,2.219821,41.477422,32.447987,83.168830,81.325121,2.79,6.36,291.36,1.54,21.0,9.938850e+08,Asia
2312,China,1982,0.781921,2.409641,42.131662,33.516429,84.117660,83.870243,2.97,6.60,281.50,1.69,21.1,1.008630e+09,Asia
2313,China,1983,0.922882,2.599462,42.785902,34.584871,85.066490,86.415364,2.56,6.90,297.81,1.40,21.1,1.023310e+09,Asia
2314,China,1984,1.063843,2.789283,43.440142,35.653313,86.015320,88.960486,2.61,6.82,302.59,1.49,21.3,1.036825e+09,Asia
2315,China,1985,1.204804,2.979104,46.352287,39.342209,86.964150,91.505607,2.63,6.78,294.79,1.57,21.7,1.051040e+09,Asia
2316,China,1986,1.345764,3.168924,47.139284,40.528096,87.912980,94.050729,2.72,6.86,281.70,1.72,22.1,1.066790e+09,Asia
2317,China,1987,1.486725,3.358745,47.477936,41.286927,88.861810,96.595850,2.76,6.72,301.73,1.81,22.5,1.084035e+09,Asia
2318,China,1988,1.627686,3.548566,47.907474,42.172798,90.651945,98.105025,2.54,6.64,370.22,1.67,22.9,1.101630e+09,Asia
2319,China,1989,1.768646,3.738386,48.375144,43.072016,92.442080,99.614200,2.52,6.54,407.89,1.70,23.3,1.118650e+09,Asia


In [23]:
filled_data

,Country Name,Year,Tertiary Enrolment (F),Tertiary Enrolment (M),Secondary Enrolment (M),Secondary Enrolment (F),Primary Enrolment (F),Primary Enrolment (M),Birth Rate,Death Rate,GDP per Capita,Natural Growth Rate,Median Age,Total Population,Region
0,Afghanistan,1980,0.500000,1.860000,13.760000,3.520000,11.000000,46.060000,7.59,24.21,291.649791,2.63,15.9,12486631.0,Asia
1,Afghanistan,1981,0.520303,2.150378,14.289395,3.848606,11.207511,45.501156,7.57,23.61,311.853617,2.67,15.8,11155195.0,Asia
2,Afghanistan,1982,0.540607,2.440757,14.818789,4.177212,11.415022,44.942312,7.55,25.62,326.366231,2.45,15.7,10088289.0,Asia
3,Afghanistan,1983,0.560910,2.731135,15.348184,4.505818,11.622532,44.383468,7.54,25.11,340.878846,2.50,15.6,9951449.0,Asia
4,Afghanistan,1984,0.581213,3.021513,15.877578,4.834425,11.830043,43.824625,7.51,30.03,355.391460,2.02,15.5,10243686.0,Asia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9235,Zimbabwe,2017,9.689270,8.005960,44.785263,44.945318,79.964715,80.780638,3.71,8.27,1192.110000,2.42,17.3,14751101.0,Africa
9236,Zimbabwe,2018,9.689270,8.005960,48.050756,48.819893,81.027363,81.907439,3.66,7.97,2269.180000,2.41,17.5,15052184.0,Africa
9237,Zimbabwe,2019,9.689270,8.005960,47.994260,48.774957,81.365767,82.355009,3.60,8.04,1421.870000,2.35,17.7,15354608.0,Africa
9238,Zimbabwe,2020,9.689270,8.005960,47.994260,48.774957,81.365767,82.355009,3.54,8.13,1372.700000,2.29,17.9,15669666.0,Africa


In [24]:
#save the data cleanly in cleaned file
filled_data.to_csv('/Users/javi/Desktop/Ironhack/Entregas_de_github/Final-Project-at-Ironhack/Data/Cleaned/MEGAMERGE_cleaned_filled_gdp.csv',index=False)

In [25]:
url = ('/Users/javi/Desktop/Ironhack/Entregas_de_github/Final-Project-at-Ironhack/Data/Cleaned/MEGAMERGE_cleaned_filled_gdp.csv')
url2 = ('/Users/javi/Desktop/Ironhack/Entregas_de_github/Final-Project-at-Ironhack/Data/Raw data/World demographic/population.csv')
# Load the provided CSV files
population_df = pd.read_csv(url2)
megamerge_df = pd.read_csv(url)

In [26]:
population_df

,Entity,Code,Year,Population (historical estimates)
0,Afghanistan,AFG,-10000,14737
1,Afghanistan,AFG,-9000,20405
2,Afghanistan,AFG,-8000,28253
3,Afghanistan,AFG,-7000,39120
4,Afghanistan,AFG,-6000,54166
...,...,...,...,...
58247,Zimbabwe,ZWE,2017,14751101
58248,Zimbabwe,ZWE,2018,15052191
58249,Zimbabwe,ZWE,2019,15354606
58250,Zimbabwe,ZWE,2020,15669663


In [27]:
megamerge_df

,Country Name,Year,Tertiary Enrolment (F),Tertiary Enrolment (M),Secondary Enrolment (M),Secondary Enrolment (F),Primary Enrolment (F),Primary Enrolment (M),Birth Rate,Death Rate,GDP per Capita,Natural Growth Rate,Median Age,Total Population,Region
0,Afghanistan,1980,0.500000,1.860000,13.760000,3.520000,11.000000,46.060000,7.59,24.21,291.649791,2.63,15.9,12486631.0,Asia
1,Afghanistan,1981,0.520303,2.150378,14.289395,3.848606,11.207511,45.501156,7.57,23.61,311.853617,2.67,15.8,11155195.0,Asia
2,Afghanistan,1982,0.540607,2.440757,14.818789,4.177212,11.415022,44.942312,7.55,25.62,326.366231,2.45,15.7,10088289.0,Asia
3,Afghanistan,1983,0.560910,2.731135,15.348184,4.505818,11.622532,44.383468,7.54,25.11,340.878846,2.50,15.6,9951449.0,Asia
4,Afghanistan,1984,0.581213,3.021513,15.877578,4.834425,11.830043,43.824625,7.51,30.03,355.391460,2.02,15.5,10243686.0,Asia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9235,Zimbabwe,2017,9.689270,8.005960,44.785263,44.945318,79.964715,80.780638,3.71,8.27,1192.110000,2.42,17.3,14751101.0,Africa
9236,Zimbabwe,2018,9.689270,8.005960,48.050756,48.819893,81.027363,81.907439,3.66,7.97,2269.180000,2.41,17.5,15052184.0,Africa
9237,Zimbabwe,2019,9.689270,8.005960,47.994260,48.774957,81.365767,82.355009,3.60,8.04,1421.870000,2.35,17.7,15354608.0,Africa
9238,Zimbabwe,2020,9.689270,8.005960,47.994260,48.774957,81.365767,82.355009,3.54,8.13,1372.700000,2.29,17.9,15669666.0,Africa


In [28]:
# Check for the missing countries in the megamerge_df
missing_countries = ['Anguilla', 'Palestine', 'Taiwan', 'Vatican']

# Filter the megamerge_df for these countries to identify the missing entries
missing_entries_megamerge = megamerge_df[megamerge_df['Country Name'].isin(missing_countries)]

# Filter the population_df for these countries to get the population data
population_entries = population_df[population_df['Entity'].isin(missing_countries)]

# Display the filtered dataframes
missing_entries_megamerge, population_entries

(     Country Name  Year  Tertiary Enrolment (F)  Tertiary Enrolment (M)  \
 252      Anguilla  1980                     NaN                     NaN   
 253      Anguilla  1981                     NaN                     NaN   
 254      Anguilla  1982                     NaN                     NaN   
 255      Anguilla  1983                     NaN                     NaN   
 256      Anguilla  1984                     NaN                     NaN   
 ...           ...   ...                     ...                     ...   
 8983      Vatican  2017                     NaN                     NaN   
 8984      Vatican  2018                     NaN                     NaN   
 8985      Vatican  2019                     NaN                     NaN   
 8986      Vatican  2020                     NaN                     NaN   
 8987      Vatican  2021                     NaN                     NaN   
 
       Secondary Enrolment (M)  Secondary Enrolment (F)  Primary Enrolment (F)  \
 252

In [29]:
# Merge the missing population data into megamerge_df based on country and year
for country in missing_countries:
    # Filter the population entries for the specific country
    country_population = population_entries[population_entries['Entity'] == country]
    
    # Iterate over the rows in megamerge_df for this country and year
    for index, row in megamerge_df[megamerge_df['Country Name'] == country].iterrows():
        year = row['Year']
        
        # Find the population for the corresponding year
        population = country_population[country_population['Year'] == year]['Population (historical estimates)']
        
        if not population.empty:
            # Update the Total Population column in megamerge_df
            megamerge_df.at[index, 'Total Population'] = population.values[0]

# Display the updated rows for verification
updated_megamerge_df = megamerge_df[megamerge_df['Country Name'].isin(missing_countries)]

In [30]:
print(updated_megamerge_df)

     Country Name  Year  Tertiary Enrolment (F)  Tertiary Enrolment (M)  \
252      Anguilla  1980                     NaN                     NaN   
253      Anguilla  1981                     NaN                     NaN   
254      Anguilla  1982                     NaN                     NaN   
255      Anguilla  1983                     NaN                     NaN   
256      Anguilla  1984                     NaN                     NaN   
...           ...   ...                     ...                     ...   
8983      Vatican  2017                     NaN                     NaN   
8984      Vatican  2018                     NaN                     NaN   
8985      Vatican  2019                     NaN                     NaN   
8986      Vatican  2020                     NaN                     NaN   
8987      Vatican  2021                     NaN                     NaN   

      Secondary Enrolment (M)  Secondary Enrolment (F)  Primary Enrolment (F)  \
252               

In [31]:
megamerge_df = megamerge_df.groupby('Country Name').apply(lambda x: x.sort_values('Year').fillna(method='ffill').fillna(method='bfill')).reset_index(drop=True)

/var/folders/23/ykjhg9v57tn3wfmhgxtgj1s80000gn/T/ipykernel_65332/3769193646.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  megamerge_df = megamerge_df.groupby('Country Name').apply(lambda x: x.sort_values('Year').fillna(method='ffill').fillna(method='bfill')).reset_index(drop=True)
/var/folders/23/ykjhg9v57tn3wfmhgxtgj1s80000gn/T/ipykernel_65332/3769193646.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  megamerge_df = megamerge_df.groupby('Country Name').apply(lambda x: x.sort_values('Year').fillna(method='ffill').fillna(method='bfill')).reset_index(drop=True)


In [32]:
megamerge_df.isna().sum()

Country Name                 0
Year                         0
Tertiary Enrolment (F)     546
Tertiary Enrolment (M)     546
Secondary Enrolment (M)    546
Secondary Enrolment (F)    546
Primary Enrolment (F)      546
Primary Enrolment (M)      546
Birth Rate                 504
Death Rate                 504
GDP per Capita             420
Natural Growth Rate          0
Median Age                   0
Total Population             0
Region                       0
dtype: int64

In [33]:
#save the data cleanly in cleaned file
megamerge_df.to_csv('/Users/javi/Desktop/Ironhack/Entregas_de_github/Final-Project-at-Ironhack/Data/Cleaned/MEGAMERGE_cleaned_filled_gdp.csv',index=False)

In [34]:
url = ('/Users/javi/Desktop/Ironhack/Entregas_de_github/Final-Project-at-Ironhack/Data/Cleaned/MEGAMERGE_cleaned_filled_gdp.csv')
megadf = pd.read_csv(url)

In [35]:
megadf

,Country Name,Year,Tertiary Enrolment (F),Tertiary Enrolment (M),Secondary Enrolment (M),Secondary Enrolment (F),Primary Enrolment (F),Primary Enrolment (M),Birth Rate,Death Rate,GDP per Capita,Natural Growth Rate,Median Age,Total Population,Region
0,Afghanistan,1980,0.500000,1.860000,13.760000,3.520000,11.000000,46.060000,7.59,24.21,291.649791,2.63,15.9,12486631.0,Asia
1,Afghanistan,1981,0.520303,2.150378,14.289395,3.848606,11.207511,45.501156,7.57,23.61,311.853617,2.67,15.8,11155195.0,Asia
2,Afghanistan,1982,0.540607,2.440757,14.818789,4.177212,11.415022,44.942312,7.55,25.62,326.366231,2.45,15.7,10088289.0,Asia
3,Afghanistan,1983,0.560910,2.731135,15.348184,4.505818,11.622532,44.383468,7.54,25.11,340.878846,2.50,15.6,9951449.0,Asia
4,Afghanistan,1984,0.581213,3.021513,15.877578,4.834425,11.830043,43.824625,7.51,30.03,355.391460,2.02,15.5,10243686.0,Asia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9235,Zimbabwe,2017,9.689270,8.005960,44.785263,44.945318,79.964715,80.780638,3.71,8.27,1192.110000,2.42,17.3,14751101.0,Africa
9236,Zimbabwe,2018,9.689270,8.005960,48.050756,48.819893,81.027363,81.907439,3.66,7.97,2269.180000,2.41,17.5,15052184.0,Africa
9237,Zimbabwe,2019,9.689270,8.005960,47.994260,48.774957,81.365767,82.355009,3.60,8.04,1421.870000,2.35,17.7,15354608.0,Africa
9238,Zimbabwe,2020,9.689270,8.005960,47.994260,48.774957,81.365767,82.355009,3.54,8.13,1372.700000,2.29,17.9,15669666.0,Africa


In [36]:
megadf.isna().sum()

Country Name                 0
Year                         0
Tertiary Enrolment (F)     546
Tertiary Enrolment (M)     546
Secondary Enrolment (M)    546
Secondary Enrolment (F)    546
Primary Enrolment (F)      546
Primary Enrolment (M)      546
Birth Rate                 504
Death Rate                 504
GDP per Capita             420
Natural Growth Rate          0
Median Age                   0
Total Population             0
Region                       0
dtype: int64

In [37]:
# Function to fill missing values with the mean of the region for that specific year
def fill_missing_with_region_mean(df):
    # List of columns to fill missing values
    columns_to_fill = [
        'Tertiary Enrolment (F)', 'Tertiary Enrolment (M)', 
        'Secondary Enrolment (M)', 'Secondary Enrolment (F)', 
        'Primary Enrolment (F)', 'Primary Enrolment (M)', 
        'Birth Rate', 'Death Rate', 'GDP per Capita', 
        'Natural Growth Rate', 'Median Age', 'Total Population'
    ]
    
    # Iterate over each year and region
    for year in df['Year'].unique():
        for region in df['Region'].unique():
            region_year_data = df[(df['Year'] == year) & (df['Region'] == region)]
            if not region_year_data.empty:
                # Calculate the mean for each column
                region_year_means = region_year_data[columns_to_fill].mean()
                # Fill missing values with the mean
                for column in columns_to_fill:
                    df.loc[(df['Year'] == year) & (df['Region'] == region), column] = \
                        df.loc[(df['Year'] == year) & (df['Region'] == region), column].fillna(region_year_means[column])
    
    return df

# Fill the missing values in megamerge_df
filled_megamerge_df = fill_missing_with_region_mean(megadf)

filled_megamerge_df.head()


,Country Name,Year,Tertiary Enrolment (F),Tertiary Enrolment (M),Secondary Enrolment (M),Secondary Enrolment (F),Primary Enrolment (F),Primary Enrolment (M),Birth Rate,Death Rate,GDP per Capita,Natural Growth Rate,Median Age,Total Population,Region
0,Afghanistan,1980,0.500000,1.860000,13.760000,3.520000,11.000000,46.060000,7.59,24.21,291.649791,2.63,15.9,12486631.0,Asia
1,Afghanistan,1981,0.520303,2.150378,14.289395,3.848606,11.207511,45.501156,7.57,23.61,311.853617,2.67,15.8,11155195.0,Asia
2,Afghanistan,1982,0.540607,2.440757,14.818789,4.177212,11.415022,44.942312,7.55,25.62,326.366231,2.45,15.7,10088289.0,Asia
3,Afghanistan,1983,0.560910,2.731135,15.348184,4.505818,11.622532,44.383468,7.54,25.11,340.878846,2.50,15.6,9951449.0,Asia
4,Afghanistan,1984,0.581213,3.021513,15.877578,4.834425,11.830043,43.824625,7.51,30.03,355.391460,2.02,15.5,10243686.0,Asia


In [38]:
filled_megamerge_df.isna().sum()

Country Name               0
Year                       0
Tertiary Enrolment (F)     0
Tertiary Enrolment (M)     0
Secondary Enrolment (M)    0
Secondary Enrolment (F)    0
Primary Enrolment (F)      0
Primary Enrolment (M)      0
Birth Rate                 0
Death Rate                 0
GDP per Capita             0
Natural Growth Rate        0
Median Age                 0
Total Population           0
Region                     0
dtype: int64

In [39]:
filtered_df_china = filled_megamerge_df[filled_megamerge_df['Country Name'] == 'China']
filtered_df_china

,Country Name,Year,Tertiary Enrolment (F),Tertiary Enrolment (M),Secondary Enrolment (M),Secondary Enrolment (F),Primary Enrolment (F),Primary Enrolment (M),Birth Rate,Death Rate,GDP per Capita,Natural Growth Rate,Median Age,Total Population,Region
1722,China,1980,0.500000,2.030000,55.110000,36.260000,82.220000,78.780000,2.74,6.34,312.02,1.47,20.8,9.812350e+08,Asia
1723,China,1981,0.640961,2.219821,41.477422,32.447987,83.168830,81.325121,2.79,6.36,291.36,1.54,21.0,9.938850e+08,Asia
1724,China,1982,0.781921,2.409641,42.131662,33.516429,84.117660,83.870243,2.97,6.60,281.50,1.69,21.1,1.008630e+09,Asia
1725,China,1983,0.922882,2.599462,42.785902,34.584871,85.066490,86.415364,2.56,6.90,297.81,1.40,21.1,1.023310e+09,Asia
1726,China,1984,1.063843,2.789283,43.440142,35.653313,86.015320,88.960486,2.61,6.82,302.59,1.49,21.3,1.036825e+09,Asia
1727,China,1985,1.204804,2.979104,46.352287,39.342209,86.964150,91.505607,2.63,6.78,294.79,1.57,21.7,1.051040e+09,Asia
1728,China,1986,1.345764,3.168924,47.139284,40.528096,87.912980,94.050729,2.72,6.86,281.70,1.72,22.1,1.066790e+09,Asia
1729,China,1987,1.486725,3.358745,47.477936,41.286927,88.861810,96.595850,2.76,6.72,301.73,1.81,22.5,1.084035e+09,Asia
1730,China,1988,1.627686,3.548566,47.907474,42.172798,90.651945,98.105025,2.54,6.64,370.22,1.67,22.9,1.101630e+09,Asia
1731,China,1989,1.768646,3.738386,48.375144,43.072016,92.442080,99.614200,2.52,6.54,407.89,1.70,23.3,1.118650e+09,Asia


In [40]:
filtered_df_spain = filled_megamerge_df[filled_megamerge_df['Country Name'] == 'Spain']
filtered_df_spain

,Country Name,Year,Tertiary Enrolment (F),Tertiary Enrolment (M),Secondary Enrolment (M),Secondary Enrolment (F),Primary Enrolment (F),Primary Enrolment (M),Birth Rate,Death Rate,GDP per Capita,Natural Growth Rate,Median Age,Total Population,Region
7770,Spain,1980,21.100000,27.220000,73.040000,75.000000,100.000000,99.990000,2.22,7.7,6208.58,0.76,29.8,37491165.0,European Union
7771,Spain,1981,22.010674,27.103812,73.717330,75.716268,99.832404,99.810191,2.04,7.8,5371.17,0.64,29.9,37758631.0,European Union
7772,Spain,1982,22.921348,26.987624,74.394660,76.432537,99.664807,99.630382,1.94,7.5,5159.71,0.60,30.1,37986012.0,European Union
7773,Spain,1983,23.832022,26.871436,75.071990,77.148805,99.497211,99.450574,1.80,7.9,4478.50,0.48,30.4,38171525.0,European Union
7774,Spain,1984,24.742696,26.755248,75.749320,77.865074,99.329615,99.270765,1.73,7.8,4489.99,0.46,30.6,38330364.0,European Union
7775,Spain,1985,25.653370,26.639060,76.426650,78.581342,99.162019,99.090956,1.64,8.1,4699.66,0.38,30.9,38469512.0,European Union
7776,Spain,1986,28.657270,28.473190,77.103980,79.297611,98.994422,98.911147,1.56,8.0,6513.50,0.34,31.2,38584624.0,European Union
7777,Spain,1987,30.226980,29.542140,77.781310,80.013879,98.826826,98.731339,1.49,8.0,8239.61,0.31,31.5,38684815.0,European Union
7778,Spain,1988,32.601610,30.947360,78.458640,80.730147,98.659230,98.551530,1.45,8.2,9703.12,0.26,31.8,38766939.0,European Union
7779,Spain,1989,34.452920,33.004150,79.135970,81.446416,99.351060,99.115590,1.40,8.4,10681.97,0.22,32.2,38827764.0,European Union


In [41]:
# Calculate metrics year by year for all countries except "World" and compare with "World"
metrics = [
    'Tertiary Enrolment (F)', 'Tertiary Enrolment (M)', 
    'Secondary Enrolment (M)', 'Secondary Enrolment (F)', 
    'Primary Enrolment (F)', 'Primary Enrolment (M)', 
    'Birth Rate', 'Death Rate', 'GDP per Capita', 
    'Natural Growth Rate', 'Median Age', 'Total Population'
]

results = []

for year in filled_megamerge_df['Year'].unique():
    # Filter data for the given year
    yearly_data = filled_megamerge_df[filled_megamerge_df['Year'] == year]
    
    # Exclude "World" region
    countries_data = yearly_data[yearly_data['Region'] != 'World']
    world_data = yearly_data[yearly_data['Region'] == 'World']
    
    num_countries = len(countries_data)
    
    # Calculate the sum for Total Population
    total_population_sum = countries_data['Total Population'].sum()
    world_population = world_data['Total Population'].values[0] if not world_data.empty else None
    
    metrics_comparison = {
        'Year': year,
        'Total Population (Sum)': total_population_sum,
        'World Population': world_population,
        'Population Percentage': (total_population_sum / world_population) * 100 if world_population else None
    }
    
    # Calculate mean for other metrics
    for metric in metrics:
        if metric != 'Total Population':
            countries_mean = countries_data[metric].sum() / num_countries
            world_mean = world_data[metric].values[0] if not world_data.empty else None
            metrics_comparison[f'{metric} (Mean)'] = countries_mean
            metrics_comparison[f'{metric} (World)'] = world_mean
            metrics_comparison[f'{metric} (Percentage)'] = (countries_mean / world_mean) * 100 if world_mean else None
    
    results.append(metrics_comparison)

comparison_df = pd.DataFrame(results)



In [42]:
float_columns =filled_megamerge_df.select_dtypes(include=['float64']).columns

# Round the float columns to 2 decimal places
filled_megamerge_df[float_columns] =filled_megamerge_df[float_columns].round(2)

In [43]:
#save the data cleanly in cleaned file
filled_megamerge_df.to_csv('/Users/javi/Desktop/Ironhack/Entregas_de_github/Final-Project-at-Ironhack/Data/Cleaned/MEGAMERGE_machine_learning_ready.csv',index=False)

In [44]:
url = ('/Users/javi/Desktop/Ironhack/Entregas_de_github/Final-Project-at-Ironhack/Data/Cleaned/MEGAMERGE_machine_learning_ready.csv')
df = pd.read_csv(url)
df

,Country Name,Year,Tertiary Enrolment (F),Tertiary Enrolment (M),Secondary Enrolment (M),Secondary Enrolment (F),Primary Enrolment (F),Primary Enrolment (M),Birth Rate,Death Rate,GDP per Capita,Natural Growth Rate,Median Age,Total Population,Region
0,Afghanistan,1980,0.50,1.86,13.76,3.52,11.00,46.06,7.59,24.21,291.65,2.63,15.9,12486631.0,Asia
1,Afghanistan,1981,0.52,2.15,14.29,3.85,11.21,45.50,7.57,23.61,311.85,2.67,15.8,11155195.0,Asia
2,Afghanistan,1982,0.54,2.44,14.82,4.18,11.42,44.94,7.55,25.62,326.37,2.45,15.7,10088289.0,Asia
3,Afghanistan,1983,0.56,2.73,15.35,4.51,11.62,44.38,7.54,25.11,340.88,2.50,15.6,9951449.0,Asia
4,Afghanistan,1984,0.58,3.02,15.88,4.83,11.83,43.82,7.51,30.03,355.39,2.02,15.5,10243686.0,Asia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9235,Zimbabwe,2017,9.69,8.01,44.79,44.95,79.96,80.78,3.71,8.27,1192.11,2.42,17.3,14751101.0,Africa
9236,Zimbabwe,2018,9.69,8.01,48.05,48.82,81.03,81.91,3.66,7.97,2269.18,2.41,17.5,15052184.0,Africa
9237,Zimbabwe,2019,9.69,8.01,47.99,48.77,81.37,82.36,3.60,8.04,1421.87,2.35,17.7,15354608.0,Africa
9238,Zimbabwe,2020,9.69,8.01,47.99,48.77,81.37,82.36,3.54,8.13,1372.70,2.29,17.9,15669666.0,Africa


In [45]:
filtered_df_spain = df[df['Country Name'] == 'World']
filtered_df_spain

,Country Name,Year,Tertiary Enrolment (F),Tertiary Enrolment (M),Secondary Enrolment (M),Secondary Enrolment (F),Primary Enrolment (F),Primary Enrolment (M),Birth Rate,Death Rate,GDP per Capita,Natural Growth Rate,Median Age,Total Population,Region
9072,World,1980,10.23,14.18,47.95,38.93,75.82,81.60,3.73,10.35,2587.77,1.79,21.5,4.442348e+09,World
9073,World,1981,10.64,14.25,48.41,39.63,75.46,82.38,3.71,10.20,2621.86,1.81,21.6,4.520917e+09,World
9074,World,1982,11.06,14.31,48.86,40.33,75.10,83.16,3.72,10.12,2538.52,1.84,21.7,4.602701e+09,World
9075,World,1983,11.47,14.37,49.32,41.03,74.74,83.94,3.59,10.17,2561.57,1.77,21.8,4.684876e+09,World
9076,World,1984,11.88,14.43,49.77,41.73,74.38,84.72,3.57,10.04,2614.18,1.78,21.9,4.766641e+09,World
9077,World,1985,12.29,14.50,50.23,42.43,74.02,85.50,3.53,9.92,2675.66,1.78,22.0,4.850077e+09,World
9078,World,1986,12.15,14.43,50.68,43.13,75.28,85.80,3.52,9.73,3109.56,1.82,22.2,4.936007e+09,World
9079,World,1987,12.27,14.52,51.14,43.83,75.79,85.92,3.49,9.56,3493.23,1.82,22.4,5.024289e+09,World
9080,World,1988,12.29,14.25,51.59,44.54,76.60,86.44,3.40,9.53,3841.11,1.77,22.6,5.113388e+09,World
9081,World,1989,12.48,14.31,52.05,45.24,77.56,86.84,3.35,9.30,3927.77,1.76,22.8,5.202583e+09,World


In [46]:
# Filter the dataframe to include only rows where Country Name is 'World'
world_data = filled_megamerge_df[filled_megamerge_df['Country Name'] == 'World']

In [47]:
# Pivot the table to make 'Year' the index and other columns as data columns
world_data_pivot = world_data.pivot(index='Year', columns='Country Name', values=world_data.columns[2:])

# Flatten the multi-index columns
world_data_pivot.columns = [f"{col[1]}_{col[0]}" for col in world_data_pivot.columns]


world_data_pivot.head()


,World_Tertiary Enrolment (F),World_Tertiary Enrolment (M),World_Secondary Enrolment (M),World_Secondary Enrolment (F),World_Primary Enrolment (F),World_Primary Enrolment (M),World_Birth Rate,World_Death Rate,World_GDP per Capita,World_Natural Growth Rate,World_Median Age,World_Total Population,World_Region
Year,,,,,,,,,,,,,
1980,10.23,14.18,47.95,38.93,75.82,81.6,3.73,10.35,2587.77,1.79,21.5,4442348279.0,World
1981,10.64,14.25,48.41,39.63,75.46,82.38,3.71,10.2,2621.86,1.81,21.6,4520917350.0,World
1982,11.06,14.31,48.86,40.33,75.1,83.16,3.72,10.12,2538.52,1.84,21.7,4602701335.0,World
1983,11.47,14.37,49.32,41.03,74.74,83.94,3.59,10.17,2561.57,1.77,21.8,4684875627.0,World
1984,11.88,14.43,49.77,41.73,74.38,84.72,3.57,10.04,2614.18,1.78,21.9,4766640881.0,World


In [48]:
# Drop the 'World_Region' column from the pivoted dataframe
world_data_pivot.drop(columns=['World_Region'], inplace=True)

In [49]:
world_data_pivot

,World_Tertiary Enrolment (F),World_Tertiary Enrolment (M),World_Secondary Enrolment (M),World_Secondary Enrolment (F),World_Primary Enrolment (F),World_Primary Enrolment (M),World_Birth Rate,World_Death Rate,World_GDP per Capita,World_Natural Growth Rate,World_Median Age,World_Total Population
Year,,,,,,,,,,,,
1980,10.23,14.18,47.95,38.93,75.82,81.6,3.73,10.35,2587.77,1.79,21.5,4442348279.0
1981,10.64,14.25,48.41,39.63,75.46,82.38,3.71,10.2,2621.86,1.81,21.6,4520917350.0
1982,11.06,14.31,48.86,40.33,75.1,83.16,3.72,10.12,2538.52,1.84,21.7,4602701335.0
1983,11.47,14.37,49.32,41.03,74.74,83.94,3.59,10.17,2561.57,1.77,21.8,4684875627.0
1984,11.88,14.43,49.77,41.73,74.38,84.72,3.57,10.04,2614.18,1.78,21.9,4766640881.0
1985,12.29,14.5,50.23,42.43,74.02,85.5,3.53,9.92,2675.66,1.78,22.0,4850076923.0
1986,12.15,14.43,50.68,43.13,75.28,85.8,3.52,9.73,3109.56,1.82,22.2,4936006502.0
1987,12.27,14.52,51.14,43.83,75.79,85.92,3.49,9.56,3493.23,1.82,22.4,5024289346.0
1988,12.29,14.25,51.59,44.54,76.6,86.44,3.4,9.53,3841.11,1.77,22.6,5113387878.0


In [50]:
#save the data cleanly in cleaned file
world_data_pivot.to_csv('/Users/javi/Desktop/Ironhack/Entregas_de_github/Final-Project-at-Ironhack/Data/Cleaned/World_machine_learning_ready.csv',index=False,encoding='utf-8')